In [67]:
from matplotlib import use
use('Qt4Agg')

import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs
from os import listdir
import cv2

In [68]:
# useful constants 
pi = np.pi
halfPi = pi/2.0
halfRangeDivPi = 255.0/pi ###WRONG
sqrtTwo = np.sqrt(2.0)
infFloat = float('inf')
sat = 0.7

In [69]:
#import image data from .emd files as type Signal2D using hyperSpy
datadir = '/Users/matt/Box/KEM Group Docs/For Matt/Datasets/180911/Processed Data/DPC/'
name = '12_DPC_1024px_1x2.5us_CL380mm 29.1 kx 0001'
emdf = hs.load(datadir+name+'.emd')

In [4]:
# Analyze .emd file tree strucutre using hyperSpy
emdf
emdf[0].original_metadata;

In [70]:
# create differential readouts in ndArray format
AC = np.empty((1024,1024), dtype=np.float32)
AC = np.subtract(emdf[4].data,emdf[0].data)      #.data attribute allows for export to ndArray

BD = np.empty((1024,1024), dtype=np.float32)
BD = np.subtract(emdf[3].data,emdf[5].data)

In [ ]:
# calculate phase using vectorized trick for quotient
# quoTrick = np.empty((1024,1024), dtype=np.float32)
# IA = np.zeros_like(AC)

# with np.nditer(IA, op_flags=['readwrite']) as it:
#     for x in it:
#         x[...] = float('inf')
        
# quoTrick = np.divide(AC, BD, out= IA, where= BD !=0)
# phaseTrick = np.arctan(quoTrick)           #how did you get this to work last night???

In [71]:
# calculate phase using laborius checking technique
quoHard = np.empty((1024,1024), dtype=np.float32)
phaseHard = np.empty((1024,1024), dtype=np.float32)

for i in range(0,1024):                                
    for j in range (0,1024):                           
            if BD[i][j] == 0:                           #handle divide by zero cases
                quoHard[i][j] = float('inf')   
            else:
                quoHard[i][j] = (AC[i][j])/(BD[i][j])   # (y/x) where AC = y, BD = x 
                
            if BD[i][j]<0:
                phaseHard[i][j] = np.arctan(quoHard[i][j])
                phaseHard[i][j] += pi
            else:
                phaseHard[i][j] = np.arctan(quoHard[i][j])
            
            phaseHard[i][j] = (phaseHard[i][j] + pi/2.0)*(179/pi)

In [ ]:
# show a difference map of the arctan of each method


In [83]:
magnitude = np.sqrt(np.add(np.square(AC),np.square(BD)))
norm = np.max(magnitude)

In [ ]:
# explore the properties of magnitude to properly normalize the dataset 
# np.max(magnitude)
# np.min(magnitude)

# for i in range(0,1024):                                
#     for j in range (0,1024): 
#         if i%100 == 0:
#             print(magnitude[i][j])

In [84]:
# normalize magnitude to fit into hsv 
#For HSV, Hue range is [0,179], Saturation range is [0,255] and Value range is [0,255]
for i in range(0,1024):                                
    for j in range (0,1024): 
        if magnitude[i][j] > 0.4*norm:
            magnitude[i][j] = 0.0
            
            
norm = np.max(magnitude)
for i in range(0,1024):                                
    for j in range (0,1024):
        magnitude[i][j]*= (1.0/norm)

In [85]:
dpcmap = np.empty((1024,1024,3),dtype=np.float32)

for i in range(0,1024):                                
    for j in range (0,1024): 
        dpcmap[i][j][0] = phaseHard[i][j]
        dpcmap[i][j][1] = 1.0
        dpcmap[i][j][2] = magnitude[i][j]

In [86]:
dpcOut = np.empty((1024,1024,3),dtype=np.float32)
dpcOut = cv2.cvtColor(dpcmap, cv2.COLOR_HSV2BGR )

In [87]:
plt.imshow(dpcOut)
plt.axis('off')
# plt.savefig(name+'_cmap.png')
# plt.savefig(name+'cmap.eps', format='eps', dpi=1000)
plt.show()